
## 1. Datasets generation
### 1.1 Generate 60*50 dataset matrix


In [1]:
import numpy as np
import matplotlib.pyplot as plt

x1 = np.random.normal(0, 1, size = (20, 50))# 20*50 matrix, which is our raw dataset where 50 is each data's dimension
x2 = np.random.normal(2, 1, size = (20, 50))
x3 = np.random.normal(4, 1, size = (20, 50))
x = np.concatenate([x1, x2], axis = 0)
x = np.concatenate([x, x3], axis = 0) #60*50 matrix, sample dataset

y = np.zeros(x.shape[0])
print(y)
print(y.shape[0])
for i in range(y.shape[0]):
    if i < y.shape[0]/3:
        y[i] = 0
    elif i >= y.shape[0]/3 and i < y.shape[0]/3*2:
        y[i] = 1
    else:
        y[i] = 2

print("number of observations:", x.shape[0])
print("number of features for each observation:", x.shape[1])
print("observations:\n", x)
print("cluster of Observations:\n", y)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
60
number of observations: 60
number of features for each observation: 50
observations:
 [[-0.06284225  1.10887859 -1.56364556 ...  2.9734763  -0.27214688
  -0.73383729]
 [-0.5004457   0.92635979 -0.30561032 ... -0.24976494 -0.48577773
  -0.71450847]
 [ 2.27092313  1.05192208  0.41315166 ...  0.62684361  0.8754012
  -0.82105893]
 ...
 [ 6.22337018  3.33705735  4.78128162 ...  5.04257052  2.86089204
   4.1865803 ]
 [ 6.03256127  4.79621909  3.57225189 ...  3.55925421  2.65694813
   4.72921091]
 [ 4.43373628  5.57526164  4.11643292 ...  5.1014171   4.86003895
   2.68850188]]
cluster of Observations:
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]


## 2. Principle Component Analysis Process




In [ ]:
def pca(x):
        v_list = np.mean(x, axis = 0)
        removedMean = x - v_list
        covariance = np.cov(x, rowvar=0)
        eigVals,eigVects = np.linalg.eig(covariance)
        #normalization
        eigVects = eigVects / np.sum(np.dot(eigVects[0].T, eigVects[0]))
        eigValsIndex = np.argsort(eigVals)
        eigValsIndex = eigValsIndex[: -3: -1] 
        v = eigVects[:,eigValsIndex]  
        finalData = np.dot(removedMean, v)
        return finalData

x_pca = pca(x)
plt.scatter(x_pca[:, 0], x_pca[:, 1], c=y, s=20)
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.show()

## 3. Application: K-means Clustering

In [3]:
from prettytable import PrettyTable

#re-label the observations
def centroid(x, y, K): 
    c_list = list()
    u_list = list()
    
    for k in range(K):
        c_list.append(np.array([x[i] for i in range(row) if y[i][0]==k]))
        u_list.append(np.mean(c_list[k], axis = 0))
    return u_list

#re-label the observations
def relabel(x, y, u_list, K):
    for i in range(row):
        dis_list = list()
        for k in range(K):
            dis_list.append(np.sum(np.square(x[i]-u_list[k])))
        index = dis_list.index(min(dis_list))
        y[i][0] = index
    return y
    
#k_mean algorithm 
def k_mean(x, y, K):
    u_list = centroid(x, y, K)
    y_ = relabel(x, y, u_list, K)
    if (y_==y).all() == False:
        k_mean(x, y_, K)
    else: 
        return y
    return y_
    
#set random seed
np.random.seed(20)
row = x.shape[0]

y_kmean_3 = np.random.randint(0, 3, size = (x.shape[0], 1))
y_new = k_mean(x, y_kmean_3, 3)

for i in range(row):
    if y_new[i][0] == 0:
        y_new[i][0] = 0
    elif y_new[i][0] == 1:
        y_new[i][0] = 2
    elif y_new[i][0] == 2:
        y_new[i][0] = 1

#compare the class
compare_table = np.zeros((3, 3))

for i in range(row):
    if y[i] == 0:
        if y_new[i][0] == 0:
            compare_table[0][0] += 1
        elif y_new[i][0] == 1:
            compare_table[0][1] += 1
        elif y_new[i][0] == 2:
            compare_table[0][2] += 1
    elif y[i] == 1:
        if y_new[i][0] == 0:
            compare_table[1][0] += 1
        elif y_new[i][0] == 1:
            compare_table[1][1] += 1
        elif y_new[i][0] == 2:
            compare_table[1][2] += 1
    elif y[i] == 2:
        if y_new[i][0] == 0:
            compare_table[2][0] += 1
        elif y_new[i][0] == 1:
            compare_table[2][1] += 1
        elif y_new[i][0] == 2:
            compare_table[2][2] += 1
         
table = PrettyTable(["# Obs.", "true_C1", "true_C2", "true_C3"])
table.add_row(["C1", int(compare_table[0][0]), int(compare_table[1][0]), int(compare_table[2][0])])
table.add_row(["C2", int(compare_table[0][1]), int(compare_table[1][1]), int(compare_table[2][1])])
table.add_row(["C3", int(compare_table[0][2]), int(compare_table[1][2]), int(compare_table[2][2])])
print(table)

#calculate the accuracy
correct = compare_table[0][0] + compare_table[1][1] + compare_table[2][2]
print("Accuracy(%): ",correct/row * 100)

+--------+---------+---------+---------+
| # Obs. | true_C1 | true_C2 | true_C3 |
+--------+---------+---------+---------+
|   C1   |    20   |    1    |    0    |
|   C2   |    0    |    12   |    0    |
|   C3   |    0    |    7    |    20   |
+--------+---------+---------+---------+
Accuracy(%):  86.66666666666667
